Step 0: Install necessary packages

In [ ]:
!pip install beautifulsoup4 requests


Step 1: Fetch the list of files from the GDELT 2.0 project's server, filter for URLs ending in 'export.CSV.zip', print these URLs, and count and print the total number of such files available.

In [ ]:
import requests  # Import the requests library to handle HTTP requests

# URL containing the list of files
url = "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"

# Send a GET request to fetch the content of the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:  # status_code 200 indicates a successful response
    # Split the content into lines
    lines = response.text.splitlines()  # Split the response text into individual lines
    
    # Filter lines that end with 'export.CSV.zip'
    csv_zip_urls = [line for line in lines if line.endswith('export.CSV.zip')]  
    # Create a list of lines that end with 'export.CSV.zip', indicating URLs of interest

    # Print or process the filtered URLs
    for url in csv_zip_urls:
        print(url.split()[-1])  # The URL is the last part after splitting by whitespace
        # Split each line by whitespace and print the last part, which is the actual URL of the CSV.zip file
else:
    print("Failed to fetch data:", response.status_code)  # Print an error message if the request failed

# Count the number of CSV.zip files available
num_files = len(csv_zip_urls)
print(f"Number of available CSV.zip files: {num_files}")  # Print the count of available CSV.zip files

## Number of available CSV.zip files: 318544


Step 2: Fetch the list of files from the GDELT 2.0 project's server, filter for URLs ending in 'export.CSV.zip', download these files, and save them to a specified directory on the local machine, while also providing a progress count of the total and successfully downloaded files.

In [ ]:
import requests  # Import the requests library to handle HTTP requests
import os  # Import the os library to handle file and directory operations

# URL containing the list of files
url = "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"

# Path to save the downloaded files, correctly expanding the home directory
save_path = os.path.expanduser("~/Desktop/GHVT6_Thesis/GHVT6_Data/GDELT 2.0")

# Ensure the directory exists
os.makedirs(save_path, exist_ok=True)
# os.makedirs creates the specified directory, including any necessary but nonexistent parent directories.
# The exist_ok parameter prevents raising an exception if the directory already exists.

# Start the process
print("Starting the download process...")

# Send a GET request to fetch the content of the URL
response = requests.get(url)
# This sends an HTTP GET request to the specified URL and stores the server's response in the response variable.

if response.status_code == 200:  # Check if the request was successful (status code 200)
    lines = response.text.splitlines()  # Split the response text into individual lines
    csv_zip_urls = [line for line in lines if line.endswith('export.CSV.zip')]  
    # Create a list of lines that end with 'export.CSV.zip', indicating URLs of interest
    
    total_files = len(csv_zip_urls)  # Get the total number of files to download
    downloaded_files = 0  # Initialise a counter for the number of successfully downloaded files

    for line in csv_zip_urls:
        file_url = line.split()[-1]  # Extract the URL (the last part after splitting by whitespace)
        filename = file_url.split('/')[-1]  # Extract the filename from the URL
        file_path = os.path.join(save_path, filename)  # Create the full path for saving the file

        print(f"Downloading {file_url}...")  # Print the URL of the file being downloaded
        r = requests.get(file_url)  # Send a GET request to download the file
        if r.status_code == 200:  # Check if the file was downloaded successfully
            with open(file_path, 'wb') as f:  # Open the file in write-binary mode
                f.write(r.content)  # Write the content of the response to the file
            downloaded_files += 1  # Increment the counter of successfully downloaded files
            print(f"Saved to {file_path} ({downloaded_files}/{total_files})")  
            # Print a message indicating the file was saved and show the progress
        else:
            print(f"Failed to download {file_url}. Status code: {r.status_code}")  
            # Print an error message if the file could not be downloaded

    print(f"Download process completed. {downloaded_files} out of {total_files} files were downloaded successfully.")
    # Print a completion message showing the number of files successfully downloaded
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    # Print an error message if the initial request to fetch the list of files failed

Step 3: Process ZIP files containing CSV data, extract information from specific columns and save the filtered data to new CSV files in batches

In [ ]:
import pandas as pd  # Import pandas for data manipulation
import zipfile  # Import zipfile to handle zip files
import os  # Import os to handle file and directory operations
import csv  # Import csv to handle CSV file operations
import logging  # Import logging to enable logging

# Set up basic logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Define the directory containing your ZIP files and the output directory
input_directory = os.path.expanduser('~/Desktop/GHVT6_Thesis/GHVT6_Data/GDELT 2.0')
output_directory = os.path.expanduser('~/Desktop/GHVT6_Thesis/GHVT6_Data/GDELT 2.0/Filtered GDELT 2.0 Data')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)  # Create the output directory 

batch_size = 10  # Adjust this based on how many files you want to process at a time

def process_csv_entries(zip_file, filename):
 
    ## Generator that yields selected data from a CSV file within a zip archive.
    
    ## Args:
    ##    zip_file (ZipFile): The zipfile object containing the CSV file.
    ##    filename (str): The name of the CSV file inside the zip archive.
    
    ## Yields:
    ##     dict: A dictionary with selected data from the CSV file.
   
    with zip_file.open(filename, mode='r') as file:
        reader = csv.reader((line.decode('utf-8') for line in file), delimiter='\t')  
        for row in reader:
            if len(row) >= 58:  # Ensure the row has enough columns
                yield {
                    'year_month_day': row[1],
                    'cameo_code': row[26],
                    'goldstein_scale': row[30],
                    'actor1_full_name': row[36],
                    'actor1': row[37],
                    'actor2_full_name': row[44],
                    'actor2': row[45]
                }

def process_batch(zip_files):
    
    ## Process a batch of zip files and return a DataFrame of the extracted data.
    
    ## Args:
    ##    zip_files (list): List of zip file names to process.
    
    ## Returns:
    ##    DataFrame: A pandas DataFrame containing the extracted data.
    
    data = []
    for zip_filename in zip_files:
        zip_path = os.path.join(input_directory, zip_filename)
        logging.info(f"Processing ZIP file: {zip_path}")
        with zipfile.ZipFile(zip_path, 'r') as z:
            for csv_filename in z.namelist():
                logging.info(f"Processing CSV file: {csv_filename}")
                entries = process_csv_entries(z, csv_filename)
                data.extend(entries)
    return pd.DataFrame(data)

# Process ZIP files in batches
zip_files = [f for f in os.listdir(input_directory) if f.endswith('.zip')]
for i in range(0, len(zip_files), batch_size):
    batch_files = zip_files[i:i + batch_size]
    logging.info(f"Starting batch {i//batch_size + 1}/{(len(zip_files) + batch_size - 1)//batch_size}")
    df_batch = process_batch(batch_files)
    if not df_batch.empty:
        output_file_path = os.path.join(output_directory, f'filtered_data_batch_{i//batch_size + 1}.csv')
        df_batch.to_csv(output_file_path, index=False)
        logging.info(f"Batch {i//batch_size + 1} saved successfully at {output_file_path}.")
    else:
        logging.warning(f"No data found in batch {i//batch_size + 1}.")

logging.info("Data extraction complete.")

Step 4: Process CSV files to replace country initials with full names, filter rows based on specific countries, and save the filtered data to new CSV files while maintaining a log of processed files.

In [ ]:
import os  # Import os to handle file and directory operations
import pandas as pd  # Import pandas for data manipulation
from tqdm import tqdm  # Import tqdm for progress bar

# Define the directory with CSV files
directory = os.path.expanduser('~/Desktop/GHVT6_Thesis/GHVT6_Data/GDELT 2.0/Filtered GDELT 2.0 Data')
output_directory = os.path.expanduser('~/Desktop/GHVT6_Thesis/GHVT6_Data/GDELT 2.0/Filtered GDELT 2.0 Data/Processed GDELT 2.0 Data')  # Output directory for filtered files

# Ensure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    # Create the output directory if it doesn't exist

log_path = os.path.join(directory, 'processed_files.log')
# Define the path to the log file that keeps track of processed files

# Country initials and their mappings
country_initials = {
    'AF': 'Afghanistan', 'AL': 'Albania', 'BG': 'Bangladesh', 'CM': 'Cameroon',
    'CG': 'Democratic Republic of the Congo', 'EG': 'Egypt', 'ER': 'Eritrea',
    'GG': 'Georgia', 'IN': 'India', 'IR': 'Iran', 'IZ': 'Iraq', 
    'PK': 'Pakistan', 'WE': 'Palestine', 'GZ': 'Palestine', 'SL': 'Sierra Leone', 'SO': 'Somalia',
    'SY': 'Syria', 'TU': 'Türkiye', 'YM': 'Yemen'
}
# Dictionary mapping country initials to their full names

# Read in existing log entries, if any
processed_files = set()
if os.path.exists(log_path):
    with open(log_path, 'r') as file:
        processed_files.update(file.read().splitlines())
        # Read previously processed files from the log and add them to the set

# Process each file in the directory
files = [f for f in os.listdir(directory) if f.endswith('.csv') and f not in processed_files]
# List all CSV files in the directory that haven't been processed yet

for file in tqdm(files, desc="Processing files"):
    # Iterate over the unprocessed CSV files with a progress bar
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        # Read the CSV file into a DataFrame

        # Replace country initials with full names
        df['actor1'] = df['actor1'].replace(country_initials)
        df['actor2'] = df['actor2'].replace(country_initials)

        # Filter rows where either actor1 or actor2 is in the list of countries of interest
        df_filtered = df[(df['actor1'].isin(country_initials.values())) | (df['actor2'].isin(country_initials.values()))]
        
        if not df_filtered.empty:
            # If the filtered DataFrame is not empty, save it to a new CSV file in the output directory
            interim_filename = f"{os.path.splitext(file)[0]}_filtered.csv"
            interim_path = os.path.join(output_directory, interim_filename)
            df_filtered.to_csv(interim_path, index=False)

        # Log this file as processed
        with open(log_path, 'a') as log_file:
            log_file.write(file + '\n')
            # Append the file name to the log

    except Exception as e:
        print(f"Error processing file {file}: {e}")
        # Print an error message if processing fails


Step 5: Calculate the Push Factor Index

In [ ]:
import os  # Import os to handle file and directory operations
import pandas as pd  # Import pandas for data manipulation
from tqdm import tqdm  # Import tqdm for progress bar

# Define the folder containing the CSV files
input_folder = '~/Desktop/GHVT6_Thesis/GHVT6_Data/GDELT 2.0/Filtered GDELT 2.0 Data/Processed GDELT 2.0 Data'
output_folder = '~/Desktop/GHVT6_Thesis/GHVT6_Data/GDELT 2.0/Filtered GDELT 2.0 Data/Processed GDELT 2.0 Data/Processed_Outputs'
os.makedirs(os.path.expanduser(output_folder), exist_ok=True)
# Ensure the output folder exists, create it if it doesn't

# Define the country codes and names for lookup
country_initials = {
    'AF': 'Afghanistan', 'AL': 'Albania', 'BG': 'Bangladesh', 'CM': 'Cameroon',
    'CG': 'Democratic Republic of the Congo', 'EG': 'Egypt', 'ER': 'Eritrea',
    'GG': 'Georgia', 'IN': 'India', 'IR': 'Iran', 'IZ': 'Iraq', 
    'PK': 'Pakistan', 'WE': 'Palestine', 'GZ': 'Palestine', 'SL': 'Sierra Leone', 'SO': 'Somalia',
    'SY': 'Syria', 'TU': 'Türkiye', 'YM': 'Yemen'
}
# Dictionary mapping country initials to their full names

# Reverse mapping to allow lookup by both code and full name
country_lookup = {**country_initials, **{v: v for v in country_initials.values()}}

# Define the CAMEO codes with their weights and categories
events_info = {
    '110': {'weight': 0, 'category': 'Other', 'sign': 'No'},
    '111': {'weight': 0, 'category': 'Other', 'sign': 'No'},
    '112': {'weight': 0, 'category': 'Other', 'sign': 'No'},
    '232': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '234': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '254': {'weight': 1, 'category': 'Economic', 'sign': '+'},
    '255': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '25': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '20': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '27': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '28': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '354': {'weight': -1, 'category': 'Economic', 'sign': '-'},
    '355': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '350': {'weight': -1, 'category': 'Conflict', 'sign': '-'},
    '30': {'weight': -1, 'category': 'Conflict', 'sign': '-'},
    '37': {'weight': -1, 'category': 'Conflict', 'sign': '-'},
    '38': {'weight': -1, 'category': 'Conflict', 'sign': '-'},
    '81': {'weight': -1, 'category': 'Conflict', 'sign': '-'},
    '871': {'weight': -1, 'category': 'Conflict', 'sign': '-'},
    '872': {'weight': -1, 'category': 'Conflict', 'sign': '-'},
    '873': {'weight': -2, 'category': 'Conflict', 'sign': '-'},
    '874': {'weight': -3, 'category': 'Conflict', 'sign': '-'},
    '93': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '94': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '1012': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '1014': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '1032': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '1034': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '1054': {'weight': 2, 'category': 'Economic', 'sign': '+'},
    '1055': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '1050': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '100': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '107': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '108': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '1123': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '1124': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '1125': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '1244': {'weight': 2, 'category': 'Economic', 'sign': '+'},
    '1245': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '1240': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '125': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '120': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '127': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '138': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '1382': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '1383': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '1384': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '1385': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '139': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '150': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '152': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '154': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '155': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '104': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '105': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '100': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '101': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '102': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '103': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '170': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '171': {'weight': 1, 'category': 'Social', 'sign': '+'},
    '180': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '190': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '191': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
    '192': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '193': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '194': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '195': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '1951': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '1952': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '190': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '200': {'weight': 4, 'category': 'Conflict', 'sign': '+'},
    '204': {'weight': 4, 'category': 'Conflict', 'sign': '+'},
    '2041': {'weight': 4, 'category': 'Conflict', 'sign': '+'},
    '2042': {'weight': 4, 'category': 'Conflict', 'sign': '+'},
    '23': {'weight': 2, 'category': 'Economic', 'sign': '+'},
    '231': {'weight': 2, 'category': 'Economic', 'sign': '+'},
    '1011': {'weight': 2, 'category': 'Economic', 'sign': '+'},
    '103': {'weight': 2, 'category': 'Economic', 'sign': '+'},
    '1031': {'weight': 2, 'category': 'Economic', 'sign': '+'},
    '24': {'weight': 1, 'category': 'Governance', 'sign': '+'},
    '241': {'weight': 1, 'category': 'Governance', 'sign': '+'}, 

'244': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'34': {'weight': -1, 'category': 'Governance', 'sign': '-'},
'341': {'weight': -1, 'category': 'Governance', 'sign': '-'},
'342': {'weight': -1, 'category': 'Governance', 'sign': '-'},
'344': {'weight': -1, 'category': 'Governance', 'sign': '-'},
'35': {'weight': -1, 'category': 'Governance', 'sign': '-'},
'83': {'weight': -2, 'category': 'Governance', 'sign': '-'},
'831': {'weight': -2, 'category': 'Governance', 'sign': '-'},
'832': {'weight': -2, 'category': 'Governance', 'sign': '-'},
'834': {'weight': -2, 'category': 'Governance', 'sign': '-'},
'91': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'104': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'1041': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'1042': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'1044': {'weight': -1, 'category': 'Governance', 'sign': '-'},
'105': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'1121': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'123': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'1231': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'1232': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'1234': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'124': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'1241': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'128': {'weight': 1, 'category': 'Governance', 'sign': '+'},
'243': {'weight': 1, 'category': 'Political', 'sign': '+'},
'251': {'weight': 1, 'category': 'Political', 'sign': '+'},
'253': {'weight': 1, 'category': 'Political', 'sign': '+'},
'343': {'weight': -1, 'category': 'Political', 'sign': '-'},
'351': {'weight': -1, 'category': 'Political', 'sign': '-'},
'353': {'weight': -1, 'category': 'Political', 'sign': '-'},
'75': {'weight': -2, 'category': 'Political', 'sign': '-'},

'811': {'weight': -2, 'category': 'Political', 'sign': '-'},
'812': {'weight': -2, 'category': 'Political', 'sign': '-'},
'813': {'weight': -2, 'category': 'Political', 'sign': '-'},
'814': {'weight': -2, 'category': 'Political', 'sign': '-'},
'833': {'weight': -2, 'category': 'Political', 'sign': '-'},
'92': {'weight': 1, 'category': 'Political', 'sign': '+'},
'1043': {'weight': 1, 'category': 'Political', 'sign': '+'},
'1051': {'weight': 1, 'category': 'Political', 'sign': '+'},
'1053': {'weight': 1, 'category': 'Political', 'sign': '+'},
'1122': {'weight': 1, 'category': 'Political', 'sign': '+'},
'1233': {'weight': 1, 'category': 'Political', 'sign': '+'},
'1243': {'weight': 2, 'category': 'Political', 'sign': '+'},
'1322': {'weight': 1, 'category': 'Political', 'sign': '+'},
'1323': {'weight': 1, 'category': 'Political', 'sign': '+'},
'1324': {'weight': 1, 'category': 'Political', 'sign': '+'},
'137': {'weight': 1, 'category': 'Political', 'sign': '+'},
'151': {'weight': 2, 'category': 'Political', 'sign': '+'},
'153': {'weight': 2, 'category': 'Political', 'sign': '+'},
'1711': {'weight': 3, 'category': 'Political', 'sign': '+'},

'172': {'weight': 3, 'category': 'Political', 'sign': '+'},
'1721': {'weight': 3, 'category': 'Political', 'sign': '+'},
'1722': {'weight': 3, 'category': 'Political', 'sign': '+'},
'1723': {'weight': 3, 'category': 'Political', 'sign': '+'},
'1724': {'weight': 3, 'category': 'Political', 'sign': '+'},
'173': {'weight': 3, 'category': 'Political', 'sign': '+'},
'174': {'weight': 3, 'category': 'Political', 'sign': '+'},
'175': {'weight': 3, 'category': 'Political', 'sign': '+'},
'181': {'weight': 3, 'category': 'Political', 'sign': '+'},
'1822': {'weight': 3, 'category': 'Political', 'sign': '+'},
'1823': {'weight': 3, 'category': 'Political', 'sign': '+'},
'201': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
'202': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '203': {'weight': 3, 'category': 'Conflict', 'sign': '+'},
    '233': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
    '252': {'weight': 1, 'category': 'Social', 'sign': '+'},
    '352': {'weight': -1, 'category': 'Social', 'sign': '-'},
    '82': {'weight': -2, 'category': 'Social', 'sign': '-'},
    '84': {'weight': -2, 'category': 'Political', 'sign': '-'},
    '841': {'weight': -2, 'category': 'Political', 'sign': '-'},
    '842': {'weight': -2, 'category': 'Political', 'sign': '-'},
    '1052': {'weight': 1, 'category': 'Political', 'sign': '+'},
    '113': {'weight': 2, 'category': 'Social', 'sign': '+'},
    '1242': {'weight': 2, 'category': 'Social', 'sign': '+'},
    '133': {'weight': 1, 'category': 'Social', 'sign': '+'},
    '1381': {'weight': 1, 'category': 'Social', 'sign': '+'},
    '140': {'weight': 1, 'category': 'Social', 'sign': '+'},
    '141': {'weight': 1, 'category': 'Social', 'sign': '+'},
    '1411': {'weight': 2, 'category': 'Social', 'sign': '+'},
    '1412': {'weight': 1, 'category': 'Social', 'sign': '+'},
    '1413': {'weight': 1, 'category': 'Social', 'sign': '+'},
    '1414': {'weight': 2, 'category': 'Social', 'sign': '+'},
    '142': {'weight': 1, 'category': 'Social', 'sign': '+'},
    '1421': {'weight': 2, 'category': 'Social', 'sign': '+'},
    '1422': {'weight': 1, 'category': 'Social', 'sign': '+'},

'1423': {'weight': 1, 'category': 'Social', 'sign': '+'},
'1424': {'weight': 2, 'category': 'Social', 'sign': '+'},
'143': {'weight': 1, 'category': 'Social', 'sign': '+'},
'1431': {'weight': 2, 'category': 'Social', 'sign': '+'},
'1432': {'weight': 1, 'category': 'Social', 'sign': '+'},
'1433': {'weight': 1, 'category': 'Social', 'sign': '+'},
'1434': {'weight': 2, 'category': 'Social', 'sign': '+'},
'144': {'weight': 1, 'category': 'Social', 'sign': '+'},
'1441': {'weight': 2, 'category': 'Social', 'sign': '+'},
'1442': {'weight': 1, 'category': 'Social', 'sign': '+'},
'1443': {'weight': 1, 'category': 'Social', 'sign': '+'},
'43': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'1444': {'weight': 2, 'category': 'Social', 'sign': '+'},
'145': {'weight': 2, 'category': 'Social', 'sign': '+'},
'1451': {'weight': 3, 'category': 'Social', 'sign': '+'},
'1452': {'weight': 3, 'category': 'Social', 'sign': '+'},
'1453': {'weight': 3, 'category': 'Social', 'sign': '+'},
'1454': {'weight': 3, 'category': 'Social', 'sign': '+'},
'171': {'weight': 1, 'category': 'Social', 'sign': '+'},
'1712': {'weight': 1, 'category': 'Social', 'sign': '+'},
'182': {'weight': 2, 'category': 'Social', 'sign': '+'},
'1821': {'weight': 1, 'category': 'Social', 'sign': '+'},
'183': {'weight': 3, 'category': 'Social', 'sign': '+'},
'1831': {'weight': 3, 'category': 'Social', 'sign': '+'},
'1832': {'weight': 3, 'category': 'Social', 'sign': '+'},
'1833': {'weight': 3, 'category': 'Social', 'sign': '+'},
'1834': {'weight': 1, 'category': 'Social', 'sign': '+'},
'184': {'weight': 3, 'category': 'Social', 'sign': '+'},
'185': {'weight': 2, 'category': 'Social', 'sign': '+'},
'180': {'weight': 2, 'category': 'Social', 'sign': '+'},
'12': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'13': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
'100': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
'1': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'10': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'11': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'14': {'weight': 1, 'category': 'Social', 'sign': '+'},
'15': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
'10': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'11': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'12': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
'121': {'weight': 2, 'category': 'Economic', 'sign': '+'},
'122': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
'1221': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
'1222': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
'1223': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
'1224': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
'129': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'130': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'131': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'1311': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'1312': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'1313': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'132': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'1321': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'134': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'135': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'130': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'131': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'1311': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'1312': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'1313': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'132': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'1321': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'134': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'135': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'130': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'14': {'weight': 1, 'category': 'Social', 'sign': '+'},
'15': {'weight': 1, 'category': 'Conflict', 'sign': '+'},
'101': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'102': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
'1021': {'weight': 2, 'category': 'Economic', 'sign': '+'},
'1022': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
'1023': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
'103': {'weight': 2, 'category': 'Conflict', 'sign': '+'},
'17': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'18': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'19': {'weight': 0, 'category': 'Other', 'sign': 'No'},
'20': {'weight': 0, 'category': 'Other', 'sign': 'No'}

}

# Function to assign weights and categories
def assign_event_info(cameo_code):
    event = events_info.get(str(cameo_code), {'weight': 0, 'category': 'Unknown'})
    return pd.Series([event['weight'], event['category']], index=['weight', 'category'])

# Function to process each row and handle actors' country assignment
def process_row(row):
    actor1_country = country_lookup.get(row['actor1'], row['actor1'])
    actor2_country = country_lookup.get(row['actor2'], row['actor2'])
    row['country'] = actor1_country
    
    # Initialise the list of rows to return; always include the first row
    rows = [row.copy()]
    
    # If actor2 is different from actor1, duplicate the row for actor2
    if actor1_country != actor2_country:
        new_row = row.copy()
        new_row['country'] = actor2_country
        rows.append(new_row)
    
    return rows

# Initialise an empty DataFrame to store the merged results
merged_df = pd.DataFrame()

# Get the list of CSV files in the input folder
csv_files = [f for f in os.listdir(os.path.expanduser(input_folder)) if f.endswith('.csv')]

# Process each file
for csv_file in tqdm(csv_files, desc="Processing files"):
    file_path = os.path.join(os.path.expanduser(input_folder), csv_file)
    
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # Apply the row processing function
    expanded_rows = df.apply(process_row, axis=1)
    new_df = pd.concat([pd.DataFrame(rows) for rows in expanded_rows], ignore_index=True)
    
    # Drop the unnecessary columns
    new_df.drop(columns=['actor1', 'actor2', 'actor1_full_name', 'actor2_full_name'], inplace=True)
    
    # Drop duplicates if necessary and reset index
    new_df.drop_duplicates(inplace=True)
    new_df.reset_index(drop=True, inplace=True)
    
    # Apply function to assign weights and categories
    new_df[['weight', 'category']] = new_df['cameo_code'].apply(assign_event_info)
    
    # Convert 'year_month_day' to datetime to facilitate resampling
    new_df['year_month_day'] = pd.to_datetime(new_df['year_month_day'], format='%Y%m%d')
    
    # Set the datetime as the index
    new_df.set_index('year_month_day', inplace=True)
    
    # Filter out rows with countries not in the specified list
    valid_countries = list(country_codes.values()) + list(country_codes.keys())
    new_df = new_df[new_df['country'].isin(valid_countries)]
    
    # Group by country and category, resample by month, and sum the weights
    monthly_pfi = new_df.groupby(['country', 'category']).resample('M').sum()['weight'].unstack(level='category', fill_value=0)
    
    # Calculate the push_factor_index
    monthly_pfi['push_factor_index'] = monthly_pfi.sum(axis=1)
    
    # Reset index to make 'country' and 'date' columns regular columns
    monthly_pfi.reset_index(inplace=True)
    
    # Save the processed file to the output folder
    output_file_path = os.path.join(os.path.expanduser(output_folder), f'processed_{csv_file}')
    monthly_pfi.to_csv(output_file_path, index=False)
    
    # Append to the merged DataFrame
    merged_df = pd.concat([merged_df, monthly_pfi], ignore_index=True)
    
    # Save the merged results to a new CSV file after each file is processed
    merged_merged_file_path = os.path.join(os.path.expanduser(output_folder), 'merged_pfi_by_country.csv')
    merged_df.to_csv(merged_merged_file_path, index=False)

print("Processing complete. Merged file saved as 'merged_pfi_by_country.csv'.")

Step 6: Process the final CSV file to convert date columns, count non-zero values in a specific column, determine unique countries, and identify any duplicate date entries.

In [ ]:
import pandas as pd  # Import pandas for data manipulation
import os  # Import os to handle file and directory operations

# Define the directory and file path
directory_path = os.path.expanduser('~/Desktop/GHVT6_Thesis/GHVT6_Data/GDELT 2.0/Filtered GDELT 2.0 Data/Processed GDELT 2.0 Data/Processed_Outputs')
merged_file_path = os.path.join(directory_path, 'merged_pfi_by_country.csv')
# Expand the user path and sets the path to the merged CSV file

# Load the merged CSV file
merged_df = pd.read_csv(merged_file_path)
# Read the CSV file into a DataFrame

# Convert 'year_month_day' to datetime 
merged_df['year_month_day'] = pd.to_datetime(merged_df['year_month_day'], format='%Y-%m-%d')
# Convert the 'year_month_day' column to datetime format

# Count nonzero values in the 'Other' column
nonzero_count_other = (merged_df['Other'] != 0).sum()
# Count the number of non-zero values in the 'Other' column

# Count the number of unique countries
unique_countries_count = merged_df['country'].nunique()
# Count the number of unique countries in the 'country' column

# Get the list of unique countries
unique_countries = merged_df['country'].unique()
# Retrieve the list of unique countries in the 'country' column

# Find duplicate year_month_day values per country
duplicate_counts = merged_df.groupby(['country', 'year_month_day']).size().reset_index(name='count')
# Groups the DataFrame by 'country' and 'year_month_day', counts occurrences, and resets the index

duplicates_per_country = duplicate_counts[duplicate_counts['count'] > 1]
# Filter the grouped DataFrame to find duplicate entries (count > 1)

# Display the results
print(f"Number of nonzero values in the 'Other' column: {nonzero_count_other}")
print(f"Number of unique countries in the dataset: {unique_countries_count}")
print(f"Countries in the dataset: {unique_countries}")

if not duplicates_per_country.empty:
    print("Duplicate year_month_day values per country:")
    print(duplicates_per_country)
else:
    print("No duplicate year_month_day values per country found.")
# Print the results, including non-zero counts, unique countries, and any duplicate date entries per country


Step 7: Ensure data is aggregated to a monthly basis and save the results to a new CSV file.

In [ ]:
import pandas as pd  # Import pandas for data manipulation
import os  # Import os to handle file and directory operations

# Define the directory and file path
directory_path = os.path.expanduser('~/Desktop/GHVT6_Thesis/GHVT6_Data/GDELT 2.0/Filtered GDELT 2.0 Data/Processed GDELT 2.0 Data/Processed_Outputs')
merged_file_path = os.path.join(directory_path, 'merged_pfi_by_country.csv')
# Expand the user path and sets the path to the merged CSV file

# Load the merged CSV file
merged_df = pd.read_csv(merged_file_path)
# Read the CSV file into a DataFrame

# Convert 'year_month_day' to datetime
merged_df['year_month_day'] = pd.to_datetime(merged_df['year_month_day'], format='%Y-%m-%d')
# Convert the 'year_month_day' column to datetime format

# Extract year and month for aggregation
merged_df['year_month'] = merged_df['year_month_day'].dt.to_period('M')
# Extract the year and month from 'year_month_day' and creates a new 'year_month' column with monthly periods

# Aggregate the data by summing up relevant columns for each 'country' and 'year_month'
aggregated_columns = ['Conflict', 'Economic', 'Governance', 'Other', 'Political', 'Social', 'Unknown', 'push_factor_index']
# Define the columns to aggregate

monthly_aggregated_df = merged_df.groupby(['country', 'year_month'])[aggregated_columns].sum().reset_index()
# Group the DataFrame by 'country' and 'year_month', sums the specified columns, and resets the index

# Display the aggregated DataFrame
print(monthly_aggregated_df)
# Print the aggregated DataFrame

# Save the aggregated DataFrame to a CSV file
monthly_aggregated_df.to_csv(os.path.join(directory_path, 'monthly_aggregated_country.csv'), index=False)
# Save the aggregated DataFrame to a new CSV file in the specified directory
